In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from PIL import Image , ImageOps


In [3]:
train_fashionMnist = datasets.FashionMNIST(root = "data",train = True, transform= ToTensor(),download=True)
test_fashionMnist = datasets.FashionMNIST(root = "data",train = False, transform= ToTensor(), download= True)

In [4]:
test_fashionMnist[0][1]

9

In [5]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, fashionMnist):
        self.fashionMnist = fashionMnist

    def __len__(self):
        return len(self.fashionMnist)

    def __getitem__(self, index):
        x = self.fashionMnist[index]
        return x[0]
    

In [6]:
train_dataset = MyDataset(train_fashionMnist)
test_dataset = MyDataset(test_fashionMnist)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size= 32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [8]:
class Encoder(nn.Module):
    def __init__(self,encoding_dim):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),            
            nn.Linear(128, encoding_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

class Decoder(nn.Module):
    def __init__(self,encoding_dim):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim,128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784)
        )

    def forward(self, x):
        x = self.decoder(x)
        return x


In [9]:
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = Encoder(encoding_dim)
        self.decoder = Decoder(encoding_dim)

    def forward(self, x):
        x = self.flatten(x)
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [10]:
model = Autoencoder(64)

In [11]:
for batch in train_dataloader:
    output = model(batch)
    break


In [12]:
print(output.shape)

torch.Size([32, 784])


In [13]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:

epochs= 3

for epoch in range(epochs):
    loss = 0
    for batch_features in train_dataloader:
        #batch leri yeniden boyutlandırma
        batch_features = batch_features.view(-1, 784)
        #Pytorch her geriye geçişte gradyanları biriktirir, bu yüzden gradyanlar sıfırlanır
        optimizer.zero_grad()
        #train dataloaderları modele sokuyoruz, yeniden yapılandırma
        train_outputs = model(batch_features)
        #yeniden yapılandırdığımız verilerle batch dataloaderları karşılaştırarak bir kayıp elde edilir
        train_loss = criterion(train_outputs, batch_features)
        #kaybın gradyanı hesaplanır
        train_loss.backward()
        #Optimizasyon algoritmasını (Adam) kullanarak geçerli gradyana göre parametreler güncellenir
        optimizer.step()
        #mini-batch training loss epoch lossa eklenir
        loss += train_loss.item()
    
    #Bir epoch tamamlandığında, toplam kaybı train dataloader sayısına bölerek, epoch train loss hesaplanır
    loss = loss / len(train_dataloader)

    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/3, loss = 0.028021
epoch : 2/3, loss = 0.017893
epoch : 3/3, loss = 0.015757


In [29]:
image = train_outputs[1].detach().numpy().reshape(28, 28)
image = (image * 255).astype(np.uint8)
image = Image.fromarray(image)
image.save("fashion_mnist_project/resim.png")

In [16]:
model.eval()
loss2 = 0

with torch.no_grad(): #gradyanların bellekte tutulması engellenir
    for batch_features in test_dataloader:
        batch_features = batch_features.view(-1, 784)
        test_outputs = model(batch_features)
        test_loss = criterion(test_outputs, batch_features)
        loss2 += test_loss.item()

loss2 = loss2 / len(test_dataloader)
print("Test loss = {:.6f}".format(loss2))

Test loss = 0.015171


In [30]:
image = test_outputs[1].detach().numpy().reshape(28, 28)
image = (image * 255).astype(np.uint8)
image = Image.fromarray(image)
image.save("fashion_mnist_project/resim1.png")

In [18]:
#import  seaborn as sbn
#sbn.lineplot(x=range(len(train_outputs)),y=loss)

In [22]:
torch.save(model.state_dict(), 'model.pth')

In [24]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [32]:
model.eval()

RecursiveScriptModule(
  original_name=Autoencoder
  (flatten): RecursiveScriptModule(original_name=Flatten)
  (encoder): RecursiveScriptModule(
    original_name=Encoder
    (encoder): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Linear)
      (3): RecursiveScriptModule(original_name=ReLU)
      (4): RecursiveScriptModule(original_name=Linear)
      (5): RecursiveScriptModule(original_name=ReLU)
      (6): RecursiveScriptModule(original_name=Linear)
    )
  )
  (decoder): RecursiveScriptModule(
    original_name=Decoder
    (decoder): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Linear)
      (3): RecursiveScriptModule(original_name=ReLU)
      (4): RecursiveScri

In [ ]:
def train():
    pass 
if __name__=="__main__":
    train()
